In [1]:
import os
import glob
import pandas as pd 
import geopandas as gpd
import geoparquet as gpq
from utils import *
from congestion_metrics import *
from mapboxgl.viz import LinestringViz
import logging
# from simpledbf import Dbf5

/Users/tszchun.chow/Library/Caches/pypoetry/virtualenvs/akl-touch-KROpMQkL-py3.12/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [2]:
## read nodes and links
def read_nodes_and_links():
    
    msm_links_dfs = {}
    counter = 0 

    for period in PERIOD_MAP:
        period_code = PERIOD_MAP[period]
        temp_dfs = []
        for scenario in BASE_SCENARIO_MAP:
            scen_code = BASE_SCENARIO_MAP[scenario]
            model_code = str(YEAR) + str(period_code) + str(scen_code)
            for file in glob.glob(SHAPEFILE_DIR + '/*/*/*.shp'):
                if model_code in file:
                    if file.endswith('emme_links.shp'):
                        temp_dfs.append(gpd.read_file(file)[ROAD_LINKS_BASE_COLS])
                    elif file.endswith('emme_nodes.shp'):
                        # do once only
                        msm_nodes_df = gpd.read_file(file)
                        counter += 1

        msm_links_dfs[period] = pd.concat(temp_dfs).drop_duplicates(subset=['ID', 'INODE', 'JNODE'])
    
    return msm_links_dfs, msm_nodes_df

msm_links_dfs, msm_nodes_df = read_nodes_and_links()

In [3]:
def map_nodes_to_zone(msm_nodes_df):
    return msm_nodes_df.sjoin(MSM_ZONES, how='left', predicate='within')[['ID', 'MSM2018', 'Sector_Name']].rename(columns={'ID': 'NODE_ID'})

def map_links(msm_links_dfs, node_zone_map):
    mapped_links_dfs = {}
    for period in PERIOD_MAP:
        mapped_links_dfs[period] = msm_links_dfs[period].copy().merge(node_zone_map, left_on='JNODE', right_on='NODE_ID')
    return mapped_links_dfs

def filter_road_links(msm_links_dfs):
    msm_road_links_dfs = {}
    for period in PERIOD_MAP:
        temp_df = msm_links_dfs[period][msm_links_dfs[period]['TYPE'] > 10]
        msm_road_links_dfs[period] = temp_df.merge(LINK_TYPE_MAP, left_on='TYPE', right_on='Link_Type', how='left')

    return msm_road_links_dfs

node_zone_map = map_nodes_to_zone(msm_nodes_df)
msm_road_links_dfs = filter_road_links(map_links(msm_links_dfs, node_zone_map))

In [30]:
def rename_columns(suffix, original_cols = ROAD_LINKS_VOL_COLS, exceptions = ["ID"]):
    new_cols = {}
    for col in original_cols:
        if col not in exceptions:
            new_cols[col] = col + "_" + suffix
        else: 
            new_cols[col] = col 
    return new_cols

def get_road_volume_time_by_periods(links_dfs):
    link_vol_dfs = {}
    for period in PERIOD_MAP:
        period_code = PERIOD_MAP[period]
        base_df = links_dfs[period].copy()
        for scenario in SCENARIO_MAP:
            scen_code = SCENARIO_MAP[scenario]
            model_code = str(YEAR) + str(period_code) + str(scen_code)
            print("Merging for model scenario: " + model_code)
            for file in glob.glob(SHAPEFILE_DIR + '/*/*/*.shp'):
                if model_code in file:
                    if file.endswith('emme_links.shp'):
                        temp_df = gpd.read_file(file)[ROAD_LINKS_VOL_COLS].rename(columns=rename_columns(model_code, ROAD_LINKS_VOL_COLS))
                        base_df = base_df.merge(temp_df, on='ID', how='left')
                        
        link_vol_dfs[period] = base_df
        del base_df

    return link_vol_dfs

def get_road_volume_time(links_dfs):
    
    for p, period in enumerate(PERIOD_MAP):
        period_code = PERIOD_MAP[period]
        if p == 0:
            base_df = links_dfs[period].copy()
        for scenario in SCENARIO_MAP:
            scen_code = SCENARIO_MAP[scenario]
            model_code = str(YEAR) + str(period_code) + str(scen_code)
            print("Merging for model scenario: " + model_code)
            for file in glob.glob(SHAPEFILE_DIR + '/*/*/*.shp'):
                if model_code in file:
                    if file.endswith('emme_links.shp'):
                        temp_df = gpd.read_file(file)[ROAD_LINKS_VOL_COLS].rename(columns=rename_columns(model_code, ROAD_LINKS_VOL_COLS))
                        base_df = base_df.merge(temp_df, on='ID', how='left')
                        

    return base_df


msm_link_vols_df = get_road_volume_time(msm_road_links_dfs)

Merging for model scenario: 26116
Merging for model scenario: 26176
Merging for model scenario: 26150
Merging for model scenario: 26152
Merging for model scenario: 26154
Merging for model scenario: 26156
Merging for model scenario: 26158
Merging for model scenario: 26164
Merging for model scenario: 26166
Merging for model scenario: 26170
Merging for model scenario: 26172
Merging for model scenario: 26174
Merging for model scenario: 26216
Merging for model scenario: 26276
Merging for model scenario: 26250
Merging for model scenario: 26252
Merging for model scenario: 26254
Merging for model scenario: 26256
Merging for model scenario: 26258
Merging for model scenario: 26264
Merging for model scenario: 26266
Merging for model scenario: 26270
Merging for model scenario: 26272
Merging for model scenario: 26274
Merging for model scenario: 26316
Merging for model scenario: 26376
Merging for model scenario: 26350
Merging for model scenario: 26352
Merging for model scenario: 26354
Merging for mo

In [12]:
def convert_geopackage_to_geoparquet(geo_package, output_path = None):
    geo_package.to_crs("EPSG:4326", inplace=True)
    geo_package['wkt'] = geo_package['geometry'].to_wkt()
    
    geo_dataframe = gpd.GeoDataFrame()
    geo_dataframe["geometry"] = gpd.GeoSeries.from_wkt(geo_package["wkt"])    
    geo_dataframe = gpd.GeoDataFrame(geo_dataframe, geometry="geometry")
    geo_dataframe["ID"] = geo_package["ID"]
    
    return geo_dataframe.merge(geo_package[[i for i in geo_package.columns if i not in ["geometry", "wkt"]]],
                              left_on="ID", right_on = "ID")
# geo_dataframe = gpd.GeoDataFrame(geo_dataframe, geometry="geometry")

/var/folders/4x/lf34fbnd6sjcsfz9532hs7yrlpdrcj/T/ipykernel_1826/2698129563.py:3: UserWarning: Geometry column does not contain geometry.
  gdf['geometry'] = gdf['geometry'].to_wkt()


,ID,INODE,JNODE,LENGTH,TYPE,MODES,LANES,geometry,NODE_ID,MSM2018,...,VOLAX_26172,VOLAU_26172,VOLAD_26172,TIMAU_26172,@vcv_26172,VOLAX_26174,VOLAU_26174,VOLAD_26174,TIMAU_26174,@vcv_26174
0,1000-5000,1000,5000,1.448476,19,abi,5.0,"LINESTRING (174.750465 -36.82373, 174.742829 -...",5000,254.0,...,0.000000,14933.29300,0.0,1.046987,0.779810,0.000000,14140.872000,0.0,1.041857,0.738424
1,1001-1002,1001,1002,2.140000,16,abw,1.0,"LINESTRING (174.737988 -36.677441, 174.718876 ...",1002,48.0,...,0.015601,304.55286,0.0,2.592589,0.180025,0.015266,316.725310,0.0,2.593618,0.187240
2,1002-1001,1002,1001,2.140000,16,abw,1.0,"LINESTRING (174.718876 -36.689067, 174.737988 ...",1001,49.0,...,0.000000,431.07416,0.0,2.491348,0.159657,0.000000,435.301240,0.0,2.491032,0.161223
3,1002-1013,1002,1013,0.955616,16,abw,1.0,"LINESTRING (174.718876 -36.689067, 174.717039 ...",1013,48.0,...,0.000000,531.71167,0.0,1.219354,0.196930,0.000000,545.521180,0.0,1.219894,0.202045
4,1002-1863,1002,1863,1.500000,26,abw,1.0,"LINESTRING (174.718876 -36.689067, 174.719762 ...",1863,48.0,...,1.048187,215.63437,0.0,1.515292,0.107817,1.056539,217.690020,0.0,1.515456,0.108845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15354,14002-1132,14002,1132,0.060000,15,bwi,1.0,"LINESTRING (174.711275 -36.72278, 174.711583 -...",1132,74.0,...,0.000000,0.00000,0.0,-1.000000,0.000000,0.000000,0.000000,0.0,-1.000000,0.000000
15355,14002-14001,14002,14001,0.060000,15,bwi,1.0,"LINESTRING (174.711275 -36.72278, 174.710931 -...",14001,74.0,...,0.000000,0.00000,0.0,-1.000000,0.000000,0.000000,0.000000,0.0,-1.000000,0.000000
15356,5242-7334,5242,7334,0.911615,16,a,1.0,"LINESTRING (174.7743 -36.85048, 174.768501 -36...",7334,264.0,...,NaN,NaN,NaN,NaN,NaN,0.000000,400.278350,0.0,1.002011,0.138027
15357,5340-7334,5340,7334,0.513016,16,a,1.0,"LINESTRING (174.770656 -36.852959, 174.768501 ...",7334,264.0,...,NaN,NaN,NaN,NaN,NaN,0.000000,367.349820,0.0,0.563552,0.126672


In [16]:
def congested_link(row, suffix = ""):
    vcv_col = "vcv" + suffix
    if row[vcv_col] >= 0.82 and 11<= row['TYPE'] <= 17:
        return 1
    elif row[vcv_col] >= 0.9 and 18 <= row['TYPE'] <=21 :
        return 1
    elif row[vcv_col] >= 0.58 and 23 <= row['TYPE'] <= 27:
        return 1
    elif row[vcv_col] >= 0.46 and row['TYPE']==22:
        return 1
    else:
        return 0

def export_geoparquet(geo_package, output_path = None, filename = None):
    geo_package.to_crs("EPSG:4326", inplace=True)
    geo_package['wkt'] = geo_package['geometry'].to_wkt()
    
    geo_dataframe = gpd.GeoDataFrame()
    geo_dataframe["geometry"] = gpd.GeoSeries.from_wkt(geo_package["wkt"])    
    geo_dataframe = gpd.GeoDataFrame(geo_dataframe, geometry="geometry")
    geo_dataframe["ID"] = geo_package["ID"]

    geo_parquet = geo_dataframe.merge(geo_package[[i for i in geo_package.columns if i not in ["geometry", "wkt"]]],
                                    left_on="ID", right_on = "ID").drop(columns=['Sector_Name'])

    if output_path is not None and filename is not None:
        geo_parquet.fillna(0, inplace=True)
        geo_parquet.to_parquet(f'{CONGESTION_METRICS_DIR}/{filename}', compression = 'zstd')


def calculate_congestion_values(road_links_dfs, export = True):
    for period in PERIOD_MAP:
        period_code = PERIOD_MAP[period]
        
        road_links_df = road_links_dfs[period].copy()
        road_links_df['LANE_KM'] = road_links_df['LANES']*road_links_df['LENGTH']

        for scenario in SCENARIO_MAP:
            scen_code = SCENARIO_MAP[scenario]
            model_code = str(YEAR) + str(period_code) + str(scen_code)
            road_links_df = road_links_df.rename(columns = {"@vcv_" +model_code: "vcv_" + model_code})

        if export:
            if not os.path.exists(CONGESTION_METRICS_DIR):
                os.makedirs(CONGESTION_METRICS_DIR)
            road_links_df.to_file(f'{CONGESTION_METRICS_DIR}/msm_links_{period}.gpkg')

            # gpkg = gpd.read_file(f'{CONGESTION_METRICS_DIR}/msm_links_{period}.gpkg', engine='pyogrio')
            export_geoparquet(road_links_df, output_path=CONGESTION_METRICS_DIR, filename = f'msm_links_{period}.parquet')

        for scenario in SCENARIO_MAP:
            scen_code = SCENARIO_MAP[scenario]
            model_code = str(YEAR) + str(period_code) + str(scen_code)

            road_links_df['CONGESTED_' + model_code] = road_links_df.apply(lambda row: congested_link(row, "_"+ model_code), axis=1)
            road_links_df['CONG_ROAD_KM_' + model_code] = road_links_df['LENGTH']*road_links_df['CONGESTED_' + model_code]
            road_links_df['CONG_LANE_KM_' + model_code] = road_links_df['LANES']*road_links_df['LENGTH']*road_links_df['CONGESTED_' + model_code]

            road_links_df['VKT_' + model_code] = road_links_df['LENGTH'] * road_links_df['VOLAU_' + model_code]
            road_links_df['VHT_' + model_code] = road_links_df['VOLAU_' + model_code] * road_links_df['TIMAU_' + model_code] / 60
            road_links_df['CONG_VKT_' + model_code] = road_links_df['LENGTH'] * road_links_df['VOLAU_' + model_code] *road_links_df['CONGESTED_' + model_code]
            road_links_df['CONG_VHT_' + model_code] = road_links_df['VOLAU_' + model_code] * road_links_df['TIMAU_' + model_code]*road_links_df['CONGESTED_' + model_code] / 60
        
        road_links_dfs[period] = road_links_df
        
        if export:
            if not os.path.exists(CONGESTION_METRICS_DIR):
                os.makedirs(CONGESTION_METRICS_DIR)
            road_links_df.to_file(f'{CONGESTION_METRICS_DIR}/msm_links_{period}.gpkg')

    return road_links_dfs

msm_link_val_dfs = calculate_congestion_values(msm_link_vols_dfs)

/var/folders/4x/lf34fbnd6sjcsfz9532hs7yrlpdrcj/T/ipykernel_1826/1677821622.py:19: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  geo_dataframe["geometry"] = gpd.GeoSeries.from_wkt(geo_package["wkt"])
/var/folders/4x/lf34fbnd6sjcsfz9532hs7yrlpdrcj/T/ipykernel_1826/1677821622.py:19: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=

In [ ]:
def generate_congestion_metrics(road_links_dfs, export = True):
    congestion_metrics_dfs = {}
    melts = []

    for period in PERIOD_MAP:
        period_code = PERIOD_MAP[period]
        road_links_df = road_links_dfs[period].copy()

        agg_metrics = {}
        agg_metrics['LENGTH'] = 'sum'
        agg_metrics['LANE_KM'] = 'sum'

        for scenario in SCENARIO_MAP:
            scen_code = SCENARIO_MAP[scenario]
            model_code = str(YEAR) + str(period_code) + str(scen_code)

            agg_metrics.update({'VKT_' + model_code: 'sum', 'VHT_' + model_code: 'sum',
                                'CONG_ROAD_KM_' + model_code :'sum', 'CONG_LANE_KM_' + model_code :'sum',
                                'CONG_VKT_' + model_code: 'sum', 'CONG_VHT_' + model_code: 'sum'})
            
            
        congestion_metrics = road_links_df.groupby(['Group_2', 'Sector_Name']).agg(agg_metrics).reset_index()
        del road_links_df

        for scenario in SCENARIO_MAP:
            scen_code = SCENARIO_MAP[scenario]
            model_code = str(YEAR) + str(period_code) + str(scen_code)
            congestion_metrics['CONG_LENGTH_%_' + model_code] = congestion_metrics['CONG_ROAD_KM_' + model_code]/congestion_metrics['LENGTH']
            congestion_metrics['CONG_LANE_%_' + model_code] = congestion_metrics['CONG_LANE_KM_' + model_code]/congestion_metrics['LANE_KM']

            if scenario not in [x for x in BASE_SCENARIO_MAP]:
                for metric in ['VKT', 'VHT', 'CONG_ROAD_KM', 'CONG_LANE_KM', 'CONG_VKT', 'CONG_VHT', 'CONG_LENGTH_%', 'CONG_LANE_%']:
                    if scenario == 'Option 3C':
                        base_scen_code = BASE_SCENARIO_MAP['Do Minimum 3C']
                    else:
                        base_scen_code = BASE_SCENARIO_MAP['Do Minimum']
                    base_model_code = str(YEAR) + str(period_code) + str(base_scen_code)

                    congestion_metrics[f'{metric}_ABSDIFF_{model_code}'] = congestion_metrics[f'{metric}_{model_code}'] - congestion_metrics[f'{metric}_{base_model_code}']
                    congestion_metrics[f'{metric}_PERCDIFF_{model_code}'] = (congestion_metrics[f'{metric}_{model_code}'] / congestion_metrics[f'{metric}_{base_model_code}']) - 1

        congestion_metrics = congestion_metrics.rename(columns={'Group_2': 'Road Type'})
        id_vars = ['Road Type', 'Sector_Name']
        congestion_metrics_melt = pd.melt(congestion_metrics, 
                                     id_vars=id_vars,
                                     value_vars = set(congestion_metrics.columns.tolist()) - set(id_vars),
                                     var_name = 'metrics_scenario', value_name = 'value')
        
        congestion_metrics_melt['metrics'] = congestion_metrics_melt['metrics_scenario'].str.rsplit('_', n=1).str[0]
        congestion_metrics_melt['scenario'] = congestion_metrics_melt['metrics_scenario'].str.split("_").str[-1]
        congestion_metrics_melt = congestion_metrics_melt.drop(columns=['metrics_scenario'])
        melts.append(congestion_metrics_melt)

        congestion_metrics_dfs[period] = congestion_metrics
        if export:
            output_dir = '../outputs/congestion_metrics'
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            congestion_metrics.to_csv(f'{output_dir}/congestion_metrics_{period}.csv', index=None)
            congestion_metrics.to_parquet(f'{output_dir}/congestion_metrics_{period}.parquet')

    pd.concat(melts).to_csv(f'{output_dir}/congestion_metrics_summary.csv', index=None)
    return congestion_metrics_dfs

congestion_metrics_dfs = generate_congestion_metrics(msm_link_vols_dfs)

/var/folders/4x/lf34fbnd6sjcsfz9532hs7yrlpdrcj/T/ipykernel_71499/2105623720.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  congestion_metrics[f'{metric}_PERCDIFF_{model_code}'] = (congestion_metrics[f'{metric}_{model_code}'] / congestion_metrics[f'{metric}_{base_model_code}']) - 1
/var/folders/4x/lf34fbnd6sjcsfz9532hs7yrlpdrcj/T/ipykernel_71499/2105623720.py:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  congestion_metrics[f'{metric}_ABSDIFF_{model_code}'] = congestion_metrics[f'{metric}_{model_code}'] - congestion_metr

In [ ]:
congestion_metrics_dfs['AM']

,Road Type,Sector_Name,LENGTH,LANE_KM,value,metrics,scenario
0,Arterial/Expressway,CBD,28.803791,54.486497,-0.809524,CONG_LENGTH_%_PERCDIFF,26156
1,Arterial/Expressway,City Fringe,44.951301,64.981301,-0.668657,CONG_LENGTH_%_PERCDIFF,26156
2,Arterial/Expressway,East Auckland,175.173337,260.377633,-0.141340,CONG_LENGTH_%_PERCDIFF,26156
3,Arterial/Expressway,Hibiscus Coast,86.907562,97.502628,0.000000,CONG_LENGTH_%_PERCDIFF,26156
4,Arterial/Expressway,Isthmus Central,121.683134,152.152062,-0.125014,CONG_LENGTH_%_PERCDIFF,26156
...,...,...,...,...,...,...,...
10747,Rural,North Shore,61.121405,61.221405,0.060008,VKT_PERCDIFF,26164
10748,Rural,Other Areas,70.873168,71.173168,-0.011324,VKT_PERCDIFF,26164
10749,Rural,Rodney,535.137776,535.137776,0.011663,VKT_PERCDIFF,26164
10750,Rural,South Auckland,440.931454,444.371454,0.008134,VKT_PERCDIFF,26164
